In [1]:
# Package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Transcript/ENSEMBL imports
conv_trans = pd.read_csv('jupyter/transcript-gene-conversion.txt', index_col = 2)
conv_id = pd.read_csv('jupyter/ensembl-geneid-conversion.txt', sep='\t', index_col = 1)

# Convert ENSEMBL IDs to genes
# Create a list of all of the transcripts to convert
transcripts = list(dat_test.index)
transcripts = [x[:-2] for x in transcripts]
transcripts_list = list(conv_trans.index)

# Replace ENSEMBL IDs with gene names, establishing the new index for all datasets
n = len(transcripts)
new_index = ['NaN']*n
for i in range(n):
    if (transcripts[i] in transcripts_list):
        new_index[i] = conv_trans.loc[transcripts[i]]['Gene stable ID']
    if (i%10000 == 0): # For troubleshooting
        print(i)

# Export the new index 
pd.DataFrame(new_index).to_csv('jupyter/index_ensembl_gene.txt')

# Convert the ensembl gene id to the common gene id
index_in = pd.read_csv('jupyter/index_ensembl_gene.txt', index_col = 1)
index_in = list(index_in.index)
genes_list = list(conv_id.index)

n = len(index_in)
final_index = ['NaN']*n
for i in range(n):
    gene = str(index_in[i])
    if (gene in genes_list):
        final_index[i] = conv_id.loc[gene]['Approved symbol']
    if (i%10000 == 0): # For troubleshooting
        print(i)

# Export the final index
pd.DataFrame(final_index).to_csv('jupyter/index_geneid.txt')

In [3]:
# Import the gene index
index_gene = pd.read_csv('jupyter/index_geneid.txt', index_col = 1)
index_gene = list(index_gene.index)

# Define a function that takes in a dataframe in the abundance.tsv format, and outputs a dataframe that
# adds the gene ID, sums "duplicates", and keeps only tpm values
def reformat(df_in):
    df = df_in
    df['ID'] = index_gene
    df = df.dropna()
    df = df.set_index('ID')
    df = df.drop(['length','eff_length','est_counts'], axis = 1)
    
    # Fix duplicates
    df['gene'] = df.index
    df = df.groupby(['gene']).sum()
    
    # Return the results
    return(df)

def reformat_counts(df_in):
    df = df_in
    df['ID'] = index_gene
    df = df.dropna()
    df = df.set_index('ID')
    df = df.drop(['length','eff_length','tpm'], axis = 1)
    
    # Fix duplicates
    df['gene'] = df.index
    df = df.groupby(['gene']).sum()
    
    # Return the results
    return(df)

In [4]:
# Define a function that takes a sample, i.e. DTB-184, as an input, reads in the file, outputs a dataframe
# with 'DTB-184' as the TPM column name, and merges that dataframe with the dataframe produced from the 
# prior iteration. Create an output tpm and similar output counts file
output_tpm = pd.DataFrame();
output_ct = pd.DataFrame();
samples = ['DTB-021','DTB-175','DTB-184','DTB-188','PR-009','PR-028','PR-038','PR-051','PR-056','PR-058','PR-063','PR-080'];
for samp in samples:
    # Read in the tsv file
    str_in = 'result-' + samp + '/abundance.tsv'
    dat_in = pd.read_csv(str_in, sep='\t', index_col = 0)
    
    # Reformat to fix duplicates and include only tpm; rename by sample
    dat_new = reformat(dat_in)
    dat_new = dat_new.rename(columns = {'tpm':samp})

    dat_ct = reformat_counts(dat_in)
    dat_ct = dat_ct.rename(columns = {'est_counts':samp})

    # Append to the output dataframes
    output_tpm = pd.concat([output_tpm, dat_new], axis=1, sort=False)
    output_ct = pd.concat([output_ct, dat_ct], axis=1, sort=False)

# Add a row with the total counts to output_ct    
output_ct['sum'] = list(output_ct.transpose().sum())

# Make a list of genes where counts exceed an average of 100 (i.e. > 1200 total counts in 12 samples)
high_genes = list(output_ct[output_ct['sum'] >= 1200].index)

# Check that the columns are identical
test = list(output_tpm.index)
test2 = list(output_ct.index)
res = (test == test2)
print('output_tpm and output_ct have identical columns? ' + str(res))

output_tpm and output_ct have identical columns? True


In [7]:
# Filter the output_tpm matrix to only include results with sufficiently high counts
tpm_final = output_tpm.loc[high_genes]

# Make dataset for top vs bottom half, bone only
bone = ['DTB-188', 'PR-063', 'PR-038', 'PR-058', 'DTB-184', 'DTB-021', 'PR-051', 'PR-080']
tpm_bone = tpm_final[bone]
tpm_bone.to_csv('jupyter/tpm_citrate_bone.csv')

# Make the design matrix based on GaCitrate uptake, split at median
des_bone = list(tpm_bone.transpose().index)
des_citrate3 = pd.DataFrame([1,1,1,1,0,0,0,0], columns = ['ga_low'])
des_citrate3['ga_high'] = 1-des_citrate3['ga_low']
des_citrate3.index = des_bone
des_citrate3.index.name = 'samples'
des_citrate3.to_csv('jupyter/des_citrate_bone.csv')


In [7]:
des_citrate3

,ga_low,ga_high
samples,,
DTB-188,1,0
PR-009,1,0
PR-028,1,0
PR-056,1,0
DTB-184,0,1
DTB-021,0,1
PR-051,0,1
PR-080,0,1
